## Imports

In [1]:
import hashlib
import json
import uuid
import pandas as pd

from pathlib import Path

## Configuration

In [2]:
input_path = Path('./input')
affiliate_input_path = input_path / 'Affiliate Values.xlsx'
lob_input_path = input_path / 'LOB to ROR replace.xlsx'
custom_field_input_path = input_path / 'Custom Field.xlsx'
ror_input_path = input_path / 'NED RoRs.xlsx'

output_path = Path('./output')

## Read and Prepare Affiliate Data

In [3]:
affiliate_data = pd.read_excel(affiliate_input_path, dtype=str)
affiliate_data = affiliate_data.dropna(subset=['ROR'])
affiliate_data = affiliate_data.drop_duplicates(subset=['ROR'])
affiliate_data = affiliate_data.sort_values(by='ROR', key=lambda x: x.str.len(), ascending=False)
affiliate_data['HASH'] = affiliate_data['ROR'].apply(lambda x: '#'.join(hashlib.sha1(str(x).encode('utf-8')).hexdigest()))
#print(affiliate_data)

## Read and Prepare LOB Data

In [4]:
lob_data = pd.read_excel(lob_input_path, dtype=str)
lob_data = lob_data.dropna(subset=['ROR'])
lob_data = lob_data.drop_duplicates(subset=['ROR'])
lob_data = lob_data.sort_values(by='ROR', key=lambda x: x.str.len(), ascending=False)
lob_data['HASH'] = lob_data['ROR'].apply(lambda x: '#'.join(hashlib.sha1(str(x).encode('utf-8')).hexdigest()))
#print(lob_data)

## 

## Read and Prepare Custom Field Data

In [5]:
custom_field_data = pd.read_excel(custom_field_input_path, dtype=str)
custom_field_data = custom_field_data.dropna(subset=['ROR'])
custom_field_data = custom_field_data.drop_duplicates(subset=['ROR'])
custom_field_data = custom_field_data.sort_values(by='ROR', key=lambda x: x.str.len(), ascending=False)
custom_field_data['HASH'] = custom_field_data['ROR'].apply(lambda x: '#'.join(hashlib.sha1(str(x).encode('utf-8')).hexdigest()))
#print(custom_field_data)

## Read ROR Data

In [6]:
ror_data = pd.read_excel(ror_input_path, dtype=str)
structured_ror_data = []
for index, row in ror_data.iterrows():
    structured_ror_data.append({
        'OFR_NUM': row['OFR_NUM'],
        'MKT_ID': row['MKT_ID'],
        'RULE_NUM': row['RULE_NUM'],
        'RULE': row['RULE'],
        'RULE_TYPE': row['RULE_TYP'],
        'STRT_DT': row['STRT_DT'],
        'END_DT': row['END_DT'],
        'ST': row['ST'],
        'EXPRESN_NUM': row['EXPRSN_NUM'],
        'EXPRSN': row['EXPRSN'],
        'TYP': row['TYP'],
        'CUST_STAT': row['CUST_STAT'],
        'PRSNT': row['PRSNT'],
        'RULE_OF_RULE': row['RULE_OF_RULE'],
        'USR_MNGD': row['USR_MNGD'],
        'AFFILIATES': [],
        'LOBS': [],
        'CUSTOM_FIELDS': [],
        'EXPANDED_RULE_OF_RULE': row['RULE_OF_RULE']
    })
ror_data = structured_ror_data

## Process ROR Data

In [7]:
# Process Affiliates
for ror in ror_data:
    for index, row in affiliate_data.iterrows():
        if row['ROR'] in ror['EXPANDED_RULE_OF_RULE']:
            ror['AFFILIATES'].append(row['Affiliate'])
            ror['EXPANDED_RULE_OF_RULE'] = ror['EXPANDED_RULE_OF_RULE'].replace(row['ROR'], row['HASH'])

# Process LOBs
for ror in ror_data:
    for index, row in lob_data.iterrows():
        if row['ROR'] in ror['EXPANDED_RULE_OF_RULE']:
            ror['LOBS'].append(row['LOB'])
            ror['EXPANDED_RULE_OF_RULE'] = ror['EXPANDED_RULE_OF_RULE'].replace(row['ROR'], row['HASH'])

# Process Custom Fields
for ror in ror_data:
    for index, row in custom_field_data.iterrows():
        if row['ROR'] in ror['EXPANDED_RULE_OF_RULE']:
            ror['CUSTOM_FIELDS'].append(row['Custom Field'])
            ror['EXPANDED_RULE_OF_RULE'] = ror['EXPANDED_RULE_OF_RULE'].replace(row['ROR'], row['HASH'])

# Replace Affiliate Hashes with Values
for ror in ror_data:
    for index, row in affiliate_data.iterrows():
        ror['EXPANDED_RULE_OF_RULE'] = ror['EXPANDED_RULE_OF_RULE'].replace(row['HASH'], row['Affiliate'])       

# Replace LOB Hashes with Values
for ror in ror_data:
    for index, row in lob_data.iterrows():
        ror['EXPANDED_RULE_OF_RULE'] = ror['EXPANDED_RULE_OF_RULE'].replace(row['HASH'], row['LOB'])

# Replace Custom Field Hashes with Values
for ror in ror_data:
    for index, row in custom_field_data.iterrows():
        ror['EXPANDED_RULE_OF_RULE'] = ror['EXPANDED_RULE_OF_RULE'].replace(row['HASH'], row['Custom Field'])


## Write Output

In [ ]:
for ror in ror_data:
    ror_string = json.dumps(ror, indent=4)

    uuid_str = str(uuid.uuid4())
    ror_output_path = output_path / (uuid_str + '.json')

    with open(ror_output_path, "w") as ror_file:
        ror_file.write(ror_string)